# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, jaccard_score
from scipy.stats import hmean
from scipy.stats.mstats import gmean
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/evrenermis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/evrenermis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/evrenermis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:
# load data from database
engine = create_engine('sqlite:///CleanData.db')
df = pd.read_sql_table('clean_dataset',engine)
X = df["message"]
Y = df[df.columns[4:]]
category_names = df.columns[4:]

### <i>Sanity Check for Data Load</i>

In [26]:
# Check for X
print(str(type(X)))
X.head()

<class 'pandas.core.series.Series'>


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [27]:
# Check for Y
print(str(type(Y)))
Y.head()

<class 'pandas.core.frame.DataFrame'>


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Check for category_names
print(str(type(category_names)))
category_names

<class 'pandas.core.indexes.base.Index'>


Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [29]:
def tokenize(text):
    # Same tokenizer function which has been explained in NLP Pipeline Section
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### <i>Message To Reader</i>

In order to build promising machine learning pipeline, the model from
Machine Learning Pipeline - Solution: GridSearch Section has been used.


In [30]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [31]:
def build_model():
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [39]:
model = build_model()
model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### <i>Sanity Check</i>

In [40]:
df.iloc[100]["message"]

'The message might be saying that they have been stuck in the presidential palace ( pal ) since the same Tuesday ( as the quake ). They need water. The message says they are not finding a little water. No names, no number of people given.'

In [41]:
msg = []
msg.append(df.iloc[100]["message"])
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'water' 'weather_related' 'earthquake']


#### <i>Excellent ! - Results are reasonable</i>

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
# Get predictions 
y_predictions = model.predict(x_test)

In [43]:
# Sanity check for predictions
y_predictions

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
# Sanity check for dataframe version
y_predictions_df = pd.DataFrame(y_predictions, columns = y_test.columns)
y_predictions_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# Print classification report
for column in y_test.columns:
    print('....................................................\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_predictions_df[column]))

....................................................

FEATURE: related

              precision    recall  f1-score   support

           0       0.61      0.34      0.44      1534
           1       0.81      0.93      0.87      4956
           2       1.00      0.16      0.27        64

    accuracy                           0.79      6554
   macro avg       0.81      0.48      0.53      6554
weighted avg       0.77      0.79      0.76      6554

....................................................

FEATURE: request

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5431
           1       0.81      0.37      0.51      1123

    accuracy                           0.88      6554
   macro avg       0.85      0.68      0.72      6554
weighted avg       0.87      0.88      0.86      6554

....................................................

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6202
           1       0.58      0.03      0.06       352

    accuracy                           0.95      6554
   macro avg       0.76      0.51      0.52      6554
weighted avg       0.93      0.95      0.92      6554

....................................................

FEATURE: direct_report

              precision    recall  f1-score   support

           0       0.85      0.98      0.91      5287
           1       0.77      0.29      0.42      1267

    accuracy                           0.85      6554
   macro avg       0.81      0.63      0.67      6554
weighted avg       0.84      0.85      0.82      6554



In [47]:
accuracy = (y_predictions == y_test).mean().mean()
print('Overall model accuracy is:  {0:.2f}% \n'.format(accuracy * 100))

Overall model accuracy is:  94.16% 



### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
def model_improved():
    
    # GridSearch parameters have been added to improve model precision.
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
    }

    pipeline = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=2)

    return pipeline

In [36]:
model_improved = model_improved()

In [37]:
model_improved.fit(x_train,y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 48.4min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 279.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'

### <i>Message To Reader</i>

It took around **5 hours** to complete fit method with GridSearch - Be patient :) <br>
I highly recommend to use your own machine instead of Udacity Portal since it can be interrupted because of the kernel shutdown. <br>
System: Macbook Air 1.8GHz Dual Core I5 - 8GB Ram


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [40]:
# Get predictions 
y_predictions = model.predict(x_test)

In [41]:
# Sanity check for predictions
y_predictions

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [42]:
# Sanity check for dataframe version
y_predictions_df = pd.DataFrame(y_predictions, columns = y_test.columns)
y_predictions_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Print classification report
for column in y_test.columns:
    print('....................................................\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_predictions_df[column]))

....................................................

FEATURE: related

              precision    recall  f1-score   support

           0       0.63      0.35      0.45      1570
           1       0.81      0.93      0.87      4935
           2       0.60      0.06      0.11        49

    accuracy                           0.79      6554
   macro avg       0.68      0.45      0.48      6554
weighted avg       0.77      0.79      0.76      6554

....................................................

FEATURE: request

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5442
           1       0.83      0.40      0.54      1112

    accuracy                           0.88      6554
   macro avg       0.86      0.69      0.74      6554
weighted avg       0.88      0.88      0.87      6554

....................................................

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      5289
           1       0.81      0.31      0.45      1265

    accuracy                           0.85      6554
   macro avg       0.84      0.65      0.68      6554
weighted avg       0.85      0.85      0.83      6554



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

By using GridSearch we improved the model accuracy from 94.16% to 94.95%. *(see below)*

- More appropriate classifier usage
- More feature availability
- Data size increase

would help to increase model accuracy.

### 9. Export your model as a pickle file

In [48]:
# save the model to disk
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [46]:
# save the model to disk
filename = 'model_improved.pkl'
pickle.dump(model_improved, open(filename, 'wb'))

### <i>Sanity Check</i>

In [48]:
from sklearn.externals import joblib

In [49]:
model = joblib.load("model.pkl")
model_improved = joblib.load("model_improved.pkl")

In [52]:
y_predictions = model.predict(x_test)

accuracy = (y_predictions == y_test).mean().mean()
print('Overall model accuracy for "model":  {0:.2f}% \n'.format(accuracy * 100))

Overall model accuracy for "model":  94.16% 



In [53]:
y_predictions = model_improved.predict(x_test)

accuracy = (y_predictions == y_test).mean().mean()
print('Overall model accuracy for "model_improved":  {0:.2f}% \n'.format(accuracy * 100))

Overall model accuracy for "model_improved":  94.95% 



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.